# 1주차: OpenAI API를 이용한 프롬프트 엔지니어링 

## 과제

In [1]:

import json
import requests
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

def get_current_weather(latitude, longitude, unit="fahrenheit"):
    
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

messages_lst = [
    [{"role": "user", "content": "지금 서울 날씨를 섭씨로 알려줘."}],
    [{"role": "user", "content": "아무말 ㅋㅅㅋ"}]
]


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "특정 지역의 날씨를 알려줍니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                    "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
                },
                "required": ["latitude", "longitude"],
            },
        }
    }
]


for messages in messages_lst :
    response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    )

    response_dict = response.to_dict()
    message_content = response_dict["choices"][0]["message"]
    
    if "tool_calls" in message_content:
        tool_call = message_content["tool_calls"][0]
        function_name = tool_call["function"]["name"]
        arguments = json.loads(tool_call["function"]["arguments"])

        if function_name == "get_current_weather":
            latitiude = arguments["latitude"] 
            longitude = arguments["longitude"]
            unit = arguments["unit"]
            analysis_result = get_current_weather(latitiude, longitude, unit)
            print(f"현재 기온({unit}): {analysis_result:.1f}")

    else:
        print(message_content["content"])
        
    
    

현재 기온(섭씨): 15.4
ㅋㅋㅋ 아무말이 오히려 더 재밌을 때도 있죠! 어떤 주제든 편하게 이야기해요!
